<a href="https://colab.research.google.com/github/iskra3138/colab_repo/blob/master/MVTEC%ED%95%99%EC%8A%B5_TPUs_TFRecords.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TPUs in Colab&nbsp; <a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50"></a>
In this example, we'll work through training a model to classify images of
flowers on Google's lightning-fast Cloud TPUs. Our model will take as input a photo of a flower and return whether it is a daisy, dandelion, rose, sunflower, or tulip.

We use the Keras framework, new to TPUs in TF 2.1.0. Adapted from [this notebook](https://colab.research.google.com/github/GoogleCloudPlatform/training-data-analyst/blob/master/courses/fast-and-lean-data-science/07_Keras_Flowers_TPU_xception_fine_tuned_best.ipynb) by [Martin Gorner](https://twitter.com/martin_gorner).

#### License

Copyright 2019-2020 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


---


This is not an official Google product but sample code provided for an educational purpose.


## Enabling and testing the TPU

First, you'll need to enable TPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select TPU from the Hardware Accelerator drop-down

Next, we'll check that we can connect to the TPU:

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [0]:
import re
import numpy as np
from matplotlib import pyplot as plt

In [0]:
import re

BUCKET = "gs://iskra3138_mvtec_tfrecords/"  #@param {type:"string", default:"jddj"}
assert re.search(r'gs://.+', BUCKET), 'For this part, you need a GCS bucket. Head to http://console.cloud.google.com/storage and create one.'


## Input data

Our input data is stored on Google Cloud Storage. To more fully use the parallelism TPUs offer us, and to avoid bottlenecking on data transfer, we've stored our input data in TFRecord files, 230 images per file.

Below, we make heavy use of `tf.data.experimental.AUTOTUNE` to optimize different parts of input loading.

All of these techniques are a bit overkill for our (small) dataset, but demonstrate best practices for using TPUs.


In [0]:
import os

AUTO = tf.data.experimental.AUTOTUNE


IMG_WIDTH = 224 #@param {type:"integer"}
IMG_HEIGHT = 224 #@param {type:"integer"}
IMAGE_SIZE =  [IMG_HEIGHT, IMG_WIDTH]

batch_size = 8 * tpu_strategy.num_replicas_in_sync

train_fns = os.path.join(BUCKET,'train.tfrecords')
validation_fns = os.path.join(BUCKET,'valid.tfrecords')

def parse_tfrecord(example):
    features = {
        'height': tf.io.FixedLenFeature([], tf.int64),
        'width': tf.io.FixedLenFeature([], tf.int64),
        'depth': tf.io.FixedLenFeature([], tf.int64),
        'label': tf.io.FixedLenFeature([], tf.int64),
        'image_raw': tf.io.FixedLenFeature([], tf.string),
    }
    # decode the TFRecord
    example = tf.io.parse_single_example(example, features)
    
    # FixedLenFeature fields are now ready to use: exmple['size']
    # VarLenFeature fields require additional sparse_to_dense decoding
    
    label = example['label']
    label = tf.one_hot(indices=label,
                      depth=2
                      )
    image = tf.io.decode_jpeg(example['image_raw'], channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [IMG_WIDTH, IMG_HEIGHT])
    
    return image, label

def load_dataset(filenames):
  # Read from TFRecords. For optimal performance, we interleave reads from multiple files.
  records = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
  return records.map(parse_tfrecord, num_parallel_calls=AUTO)

def get_training_dataset():
  dataset = load_dataset(train_fns)

  # Create some additional training images by randomly flipping and
  # increasing/decreasing the saturation of images in the training set. 
  def data_augment(image, label):
    modified = tf.image.random_flip_left_right(image)
    modified = tf.image.random_flip_up_down(modified)
    return modified, label
  augmented = dataset.map(data_augment, num_parallel_calls=AUTO)

  # Prefetch the next batch while training (autotune prefetch buffer size).
  return augmented.repeat().shuffle(2048).batch(batch_size).prefetch(AUTO) 

training_dataset = get_training_dataset()
validation_dataset = load_dataset(validation_fns).batch(batch_size).prefetch(AUTO)

Let's take a peek at the training dataset we've created:

In [0]:
CLASSES = ['NG', 'OK']

def display_one(image, title, subplot, color):
  plt.subplot(subplot)
  plt.axis('off')
  plt.imshow(image)
  plt.title(title, fontsize=16, color=color)
  
# If model is provided, use it to generate predictions.
def display_nine(images, titles, title_colors=None):
  subplot = 331
  plt.figure(figsize=(13,13))
  for i in range(9):
    color = 'black' if title_colors is None else title_colors[i]
    display_one(images[i], titles[i], 331+i, color)
  plt.tight_layout()
  plt.subplots_adjust(wspace=0.1, hspace=0.1)
  plt.show()

def get_dataset_iterator(dataset, n_examples):
  return dataset.unbatch().batch(n_examples).as_numpy_iterator()

training_viz_iterator = get_dataset_iterator(training_dataset, 9)

In [0]:
# Re-run this cell to show a new batch of images
images, classes = next(training_viz_iterator)
class_idxs = np.argmax(classes, axis=-1) # transform from one-hot array to class number
labels = [CLASSES[idx] for idx in class_idxs]
display_nine(images*255.0, labels)

## Model

모델 compile 시, tf.keras.optimizers api를 쓰면 TPU에서는 학습이 되지 않음
- optimizer='adam' <- w/callback : working
- optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=1e-4) : working
- optimizer=tf.keras.optimizers.Adam(lr=1e-4) : not working

In [0]:
def create_model():
  pretrained_model = tf.keras.applications.ResNet101(weights='imagenet', input_shape=[*IMAGE_SIZE, 3], include_top=False)
  pretrained_model.trainable = True
  x = pretrained_model.output
  x = tf.keras.layers.GlobalAveragePooling2D()(x)
  predictions = tf.keras.layers.Dense(2, activation='softmax', name='prediction')(x)
  model = tf.keras.Model(inputs=pretrained_model.input, outputs=predictions)
  
  #optimizer='adam',
  
  model.compile(
    optimizer=tf.compat.v1.train.AdamOptimizer(learning_rate=1e-4),
    loss = 'categorical_crossentropy',
    metrics=['accuracy']
  )
  return model

with tpu_strategy.scope(): # creating the model in the TPUStrategy scope means we will train the model on the TPU
  model = create_model()
model.summary()

## Training

Calculate the number of images in each dataset. Rather than actually load the data to do so (expensive), we rely on hints in the filename. This is used to calculate the number of batches per epoch.


In [0]:
train_samples = sum(1 for _ in tf.data.TFRecordDataset(train_fns))
print (train_samples)

22098


In [0]:
train_steps = train_samples // batch_size

Actually train the model. While the first epoch will be quite a bit slower as we must XLA-compile the execution graph and load the data.

In [0]:
EPOCHS=5
history = model.fit(training_dataset, validation_data=validation_dataset,
                    steps_per_epoch=train_steps, epochs=EPOCHS)
#history = model.fit(training_dataset, validation_data=validation_dataset,
#                    steps_per_epoch=train_steps, epochs=EPOCHS, callbacks=[lr_callback])
final_accuracy = history.history["val_accuracy"][-5:]
print("FINAL ACCURACY MEAN-5: ", np.mean(final_accuracy))

Train for 345 steps
Epoch 1/5
345/345 [==============================] - 151s 437ms/step - loss: 0.0160 - accuracy: 0.9943 - val_loss: 15.8790 - val_accuracy: 0.4353
Epoch 2/5
345/345 [==============================] - 52s 152ms/step - loss: 0.0015 - accuracy: 0.9997 - val_loss: 19.3712 - val_accuracy: 0.4353
Epoch 3/5
345/345 [==============================] - 52s 152ms/step - loss: 3.8070e-05 - accuracy: 1.0000 - val_loss: 22.9056 - val_accuracy: 0.4353
Epoch 4/5
345/345 [==============================] - 53s 154ms/step - loss: 0.0074 - accuracy: 0.9987 - val_loss: 12.2873 - val_accuracy: 0.4353
Epoch 5/5
345/345 [==============================] - 53s 153ms/step - loss: 6.0836e-04 - accuracy: 0.9999 - val_loss: 0.4508 - val_accuracy: 0.7719
FINAL ACCURACY MEAN-5:  0.5026523


In [0]:
EPOCHS=5
history = model.fit(training_dataset, validation_data=validation_dataset,
                    steps_per_epoch=train_steps, epochs=EPOCHS,)
final_accuracy = history.history["val_accuracy"][-5:]
print("FINAL ACCURACY MEAN-5: ", np.mean(final_accuracy))

Train for 345 steps
Epoch 1/5
345/345 [==============================] - 80s 231ms/step - loss: 2.1361e-05 - accuracy: 1.0000 - val_loss: 0.6501 - val_accuracy: 0.8641
Epoch 2/5
345/345 [==============================] - 52s 151ms/step - loss: 1.2382e-05 - accuracy: 1.0000 - val_loss: 0.0432 - val_accuracy: 0.9936
Epoch 3/5
345/345 [==============================] - 53s 153ms/step - loss: 8.2224e-06 - accuracy: 1.0000 - val_loss: 0.0451 - val_accuracy: 0.9954
Epoch 4/5
345/345 [==============================] - 52s 152ms/step - loss: 3.8053e-06 - accuracy: 1.0000 - val_loss: 0.0435 - val_accuracy: 0.9954
Epoch 5/5
345/345 [==============================] - 54s 155ms/step - loss: 2.8234e-06 - accuracy: 1.0000 - val_loss: 0.0450 - val_accuracy: 0.9954
FINAL ACCURACY MEAN-5:  0.9687945


In [0]:
def display_training_curves(training, validation, title, subplot):
  ax = plt.subplot(subplot)
  ax.plot(training)
  ax.plot(validation)
  ax.set_title('model '+ title)
  ax.set_ylabel(title)
  ax.set_xlabel('epoch')
  ax.legend(['training', 'validation'])

plt.subplots(figsize=(10,10))
plt.tight_layout()
display_training_curves(history.history['accuracy'], history.history['val_accuracy'], 'accuracy', 211)
display_training_curves(history.history['loss'], history.history['val_loss'], 'loss', 212)

Accuracy goes up and loss goes down. Looks good!

## Predictions

Let's look at some our model's predictions next to the original images. We'll show 9 images from the validation set.

In [0]:
def img_title(label, prediction):
  # Both prediction (probabilities) and label (one-hot) are arrays with one item per class.
  class_idx = np.argmax(label, axis=-1)
  prediction_idx = np.argmax(prediction, axis=-1)
  if class_idx == prediction_idx:
    return f'{CLASSES[prediction_idx]} [correct]', 'black'
  else:
    return f'{CLASSES[prediction_idx]} [incorrect, should be {CLASSES[class_idx]}]', 'red'

def get_titles(images, labels, model):
  predictions = model.predict(images)
  titles, colors = [], []
  for label, prediction in zip(classes, predictions):
    title, color = img_title(label, prediction)
    titles.append(title)
    colors.append(color)
  return titles, colors

validation_viz_iterator = get_dataset_iterator(validation_dataset, 9)

In [0]:
# Re-run this cell to show a new batch of images
images, classes = next(validation_viz_iterator)
titles, colors = get_titles(images, classes, model)
display_nine(images*255.0, titles, colors)

## Save and re-loading our trained model

In [0]:
# We can save our model with:
model.save('model.h5')
# and reload it with:
reloaded_model = tf.keras.models.load_model('model.h5')

In [0]:
# Re-run this cell to show a new batch of images
images, classes = next(validation_viz_iterator)
titles, colors = get_titles(images, classes, reloaded_model)
display_nine_flowers(images, titles, colors)